In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

# GOES Flare Profiles for SolarNN

## Flare Profiles - Case 1
- Flare profiles determined by GOES detections and random intervals between GOES detections.

In [4]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

# Generate flare profiles
FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(flare_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in flare_indices])
    exclude_indices.update([idx - k for idx in flare_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_case1.csv", index=False)

Profiles_df.head(), Profiles_df.shape

/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/1567820224.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/1567820224.py:36: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)


(         x1        x2        x3        x4        x5        x6        x7  \
 0  0.249173  0.278074  0.289489  0.297877  0.323917  0.323846  0.293258   
 1  0.185302  0.195949  0.204914  0.203649  0.217488  0.225099  0.233949   
 2  0.071275  0.084802  0.110699  0.211815  0.258221  0.193603  0.142327   
 3  0.722010  0.677981  0.669214  0.650412  0.641111  0.633326  0.625018   
 4  0.729120  0.716180  0.701948  0.645696  0.609369  0.585797  0.576336   
 
          x8        x9       x10  ...       x24       x25       x26       x27  \
 0  0.300184  0.320395  0.401313  ...  0.600410  0.576188  0.541754  0.507117   
 1  0.248383  0.258860  0.267442  ...  0.877201  1.928971  3.298110  3.872649   
 2  0.106506  0.087165  0.077384  ...  0.351790  0.349945  0.343246  0.341172   
 3  0.647180  0.679063  0.713814  ...  0.766183  0.723078  0.695107  0.683881   
 4  0.560038  0.550306  0.534466  ...  1.558642  2.091865  2.234191  2.047342   
 
         x28       x29       x30       x31    y       

## Flare Profiles - Case 2
- Flare profiles determined by GOES detections and random intervals between wavelet detections.

In [11]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_case2.csv", index=False)

Profiles_df.head(), Profiles_df.shape

/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/3734769234.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/3734769234.py:35: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)


(         x1        x2        x3        x4        x5        x6        x7  \
 0  0.249173  0.278074  0.289489  0.297877  0.323917  0.323846  0.293258   
 1  0.185302  0.195949  0.204914  0.203649  0.217488  0.225099  0.233949   
 2  0.071275  0.084802  0.110699  0.211815  0.258221  0.193603  0.142327   
 3  0.722010  0.677981  0.669214  0.650412  0.641111  0.633326  0.625018   
 4  0.729120  0.716180  0.701948  0.645696  0.609369  0.585797  0.576336   
 
          x8        x9       x10  ...       x24       x25       x26       x27  \
 0  0.300184  0.320395  0.401313  ...  0.600410  0.576188  0.541754  0.507117   
 1  0.248383  0.258860  0.267442  ...  0.877201  1.928971  3.298110  3.872649   
 2  0.106506  0.087165  0.077384  ...  0.351790  0.349945  0.343246  0.341172   
 3  0.647180  0.679063  0.713814  ...  0.766183  0.723078  0.695107  0.683881   
 4  0.560038  0.550306  0.534466  ...  1.558642  2.091865  2.234191  2.047342   
 
         x28       x29       x30       x31    y       

## Flare Profiles - Case 3
- Flare profiles for positive cases determined by GOES detections, and slide center point of window over +/- minutes from peak of flare. Negative cases determined by wavelet detection

In [10]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

# label fluxes +/- 3 mins from peak as 1
new_flare_indices = []
for i in flare_indices:
        new_flare_indices.extend([i-3, i-2, i-1, i, i+1, i+2, i+3]) # indices of +/- 3 mins from peak + peak itself

flare_indices = new_flare_indices

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 5  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_case3.csv", index=False)

Profiles_df.head(), Profiles_df.shape

/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/3571065718.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/3571065718.py:42: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)


(         x1        x2        x3        x4        x5        x6        x7  \
 0  0.428887  0.404045  0.357289  0.337403  0.376537  0.391995  0.403353   
 1  0.339866  0.300537  0.283810  0.316727  0.329730  0.339284  0.368943   
 2  0.264036  0.249340  0.278260  0.289683  0.298077  0.324133  0.324063   
 3  0.249173  0.278074  0.289489  0.297877  0.323917  0.323846  0.293258   
 4  0.302252  0.314660  0.323778  0.352081  0.352004  0.318757  0.326285   
 
          x8        x9       x10  ...       x24       x25       x26       x27  \
 0  0.438612  0.438517  0.397098  ...  0.884744  0.846193  0.828326  0.813009   
 1  0.368862  0.334022  0.341911  ...  0.711782  0.696753  0.683870  0.656281   
 2  0.293454  0.300385  0.320609  ...  0.612130  0.600812  0.576574  0.542117   
 3  0.300184  0.320395  0.401313  ...  0.600410  0.576188  0.541754  0.507117   
 4  0.348253  0.436207  0.548378  ...  0.626288  0.588860  0.551211  0.526962   
 
         x28       x29       x30       x31    y       

## Flare Profiles - Case 4
- Flare profiles determined by GOES detections and random intervals between wavelet detections. Do not normalize flux values.

In [9]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_case4.csv", index=False)

Profiles_df.head(), Profiles_df.shape

/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_4600/676565252.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_4600/676565252.py:35: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)


(             x1            x2            x3            x4            x5  \
 0  9.470907e-08  1.056939e-07  1.100330e-07  1.132213e-07  1.231185e-07   
 1  8.144197e-08  8.612121e-08  9.006145e-08  8.950541e-08  9.558777e-08   
 2  1.213154e-07  1.443386e-07  1.884173e-07  3.605219e-07  4.395089e-07   
 3  2.379977e-07  2.234842e-07  2.205944e-07  2.143965e-07  2.113307e-07   
 4  2.851891e-07  2.801277e-07  2.745612e-07  2.525584e-07  2.383497e-07   
 
              x6            x7            x8            x9           x10  ...  \
 0  1.230916e-07  1.114654e-07  1.140978e-07  1.217800e-07  1.525363e-07  ...   
 1  9.893294e-08  1.028226e-07  1.091665e-07  1.137711e-07  1.175432e-07  ...   
 2  3.295242e-07  2.422504e-07  1.812808e-07  1.483600e-07  1.317124e-07  ...   
 3  2.087644e-07  2.060258e-07  2.133311e-07  2.238410e-07  2.352960e-07  ...   
 4  2.291295e-07  2.254290e-07  2.190542e-07  2.152476e-07  2.090519e-07  ...   
 
             x24           x25           x26          

## Flare Profiles - Case 5 NEEDS TO DO
- Flare profiles determined by GOES detections and random intervals between wavelet detections. Change time scales of flare profiles.

In [12]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_case5.csv", index=False)

Profiles_df.head(), Profiles_df.shape

ModuleNotFoundError: No module named 'tensorflow'

## Flare Profiles - Case 6
- Flare profiles determined by GOES detections, random intervals between wavelet detections, and random noise at sub A-class flux levels.

In [3]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
noise2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Goes161718_wavelet_flux_noise.csv")
designaled2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Goes161718_wavelet_designaled_flux_Aclass.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: create designaled/noise profiles
noise_indices = set(range(hist1 + 1, len(designaled2017_2022) - (hist2 + 1)))
noise_indices = random.sample(noise_indices, int(nonflareMultiple/4) * numFlares)

NoiseProfiles = np.empty((int(nonflareMultiple/4) * numFlares, cols)) # 1/4 of number of non-flare profiles are noise
NoiseProfiles[:] = np.nan

j = 0
for i in noise_indices:
    fluxes = designaled2017_2022['designaled_flux'].iloc[(i - hist1):(i + hist2 + 1)] / designaled2017_2022['designaled_flux'].iloc[i]
    NoiseProfiles[j, :-2] = fluxes.values
    NoiseProfiles[j, -2] = 0
    NoiseProfiles[j, -1] = designaled2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 6: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles, NoiseProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_case6.csv", index=False)

Profiles_df.head(), Profiles_df.shape

/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_28642/3215210952.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_28642/3215210952.py:37: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_28642/3215210952.py:52: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  noise_indices = random.sample(noise_indices, int(nonflareMultiple/4) * numFlares)


(         x1        x2        x3        x4        x5        x6        x7  \
 0  0.249173  0.278074  0.289489  0.297877  0.323917  0.323846  0.293258   
 1  0.185302  0.195949  0.204914  0.203649  0.217488  0.225099  0.233949   
 2  0.071275  0.084802  0.110699  0.211815  0.258221  0.193603  0.142327   
 3  0.722010  0.677981  0.669214  0.650412  0.641111  0.633326  0.625018   
 4  0.729120  0.716180  0.701948  0.645696  0.609369  0.585797  0.576336   
 
          x8        x9       x10  ...       x24       x25       x26       x27  \
 0  0.300184  0.320395  0.401313  ...  0.600410  0.576188  0.541754  0.507117   
 1  0.248383  0.258860  0.267442  ...  0.877201  1.928971  3.298110  3.872649   
 2  0.106506  0.087165  0.077384  ...  0.351790  0.349945  0.343246  0.341172   
 3  0.647180  0.679063  0.713814  ...  0.766183  0.723078  0.695107  0.683881   
 4  0.560038  0.550306  0.534466  ...  1.558642  2.091865  2.234191  2.047342   
 
         x28       x29       x30       x31    y       

# NOTES/To-Do
- Create a new column in the profiles that indicates what kind of case they are (flare, nonflare, noise)
- Add constant (A-class level flux or slightly less) to the noise data (it's not)

## Model Evaluation Data - Flare Injections
- Inject a select set of flare profiles into different areas of GOES light curves. This data will be used to evaluate the performance of the CNN